### What is RFM segmentation?

Begavioral customer segmentation based on three metrics:
1. Recency (R)
2. Frequency (F)
3. Monetary Value (M)

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# Reading the file from csv to the dataframe in Pandas
online = pd.read_excel("Online Retail.xlsx")

### Data preparation
DataFrame with only the latest 12 months of data

In [3]:
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### calculate the Total sum values
Multiply Quantity with price

In [4]:
online['Total_sum'] = online['Quantity'] * online['UnitPrice']
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_sum
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [5]:
# latest month data December 2010 to December 2011
print('Min:{}; Max:{}'.format(min(online.InvoiceDate),max(online.InvoiceDate)))

Min:2010-12-01 08:26:00; Max:2011-12-09 12:50:00


In [6]:
# Let's creat a hypothetical snaphot_day data as if we are doing analysis recently
snapshot_date = max(online.InvoiceDate) + dt.timedelta(days=1)

## Calculate RFM values for each customer

In [7]:
# Aggregate data on a customer level
datamart = online.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'Total_sum': 'sum'
})

In [8]:
# Rename columns for easier intrerpretation
datamart.rename(columns = {'InvoiceDate': 'Recency',
                          'InvoiceNo': 'Frequency',
                          'Total_sum': 'MonetoryValue'}, inplace=True)

# check the first rows
datamart.head()

,Recency,Frequency,MonetoryValue
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [9]:
datamart.dtypes

Recency            int64
Frequency          int64
MonetoryValue    float64
dtype: object

### Mean RFM Value 

In [10]:
np.mean(datamart)

Recency            92.047118
Frequency          93.053294
MonetoryValue    1898.459701
dtype: float64

## Building RFM segments

### Recency Quartile

In [25]:
r_labels = range(4, 0, -1)
r_quartile = pd.qcut(datamart['Recency'], q=4, labels = r_labels)
datamart = datamart.assign(R = r_quartile.values)

In [26]:
datamart.head()

,Recency,Frequency,MonetoryValue,R,F,M,RFM_Segment,RFM_Score,General_Segment
CustomerID,,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3.0,Bronze
12347.0,2,182,4310.00,4,4,4,444,12.0,Gold
12348.0,75,31,1797.24,2,2,4,224,8.0,Silver
12349.0,19,73,1757.55,3,3,4,334,10.0,Gold
12350.0,310,17,334.40,1,1,2,112,4.0,Bronze


In [13]:
datamart.to_csv('datamart_rfm.csv')

## Frequency and Monetory quartiles

In [14]:
f_lables = range(1,5)
m_labels = range(1,5)

f_quartiles = pd.qcut(datamart['Frequency'], 4, labels = f_lables)
m_quartiles = pd.qcut(datamart['MonetoryValue'], 4, labels = m_labels)

datamart = datamart.assign(F = f_quartiles.values)
datamart = datamart.assign(M = m_quartiles.values)

datamart.head()

,Recency,Frequency,MonetoryValue,R,F,M
CustomerID,,,,,,
12346.0,326,2,0.00,1,1,1
12347.0,2,182,4310.00,4,4,4
12348.0,75,31,1797.24,2,2,4
12349.0,19,73,1757.55,3,3,4
12350.0,310,17,334.40,1,1,2


## Build RFM Segment and RFM Score

In [15]:
def join_rfm(x): return str(x['R']) + str(x['F']) + str(x['M'])
datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1)
datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)

In [16]:
datamart.head()

,Recency,Frequency,MonetoryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3.0
12347.0,2,182,4310.00,4,4,4,444,12.0
12348.0,75,31,1797.24,2,2,4,224,8.0
12349.0,19,73,1757.55,3,3,4,334,10.0
12350.0,310,17,334.40,1,1,2,112,4.0


## Analyzing RFM table

### Largest RFM segments

In [17]:
datamart.groupby('RFM_Segment').size().sort_values(ascending=False)[:10]

RFM_Segment
444    471
111    392
122    209
344    206
211    181
333    176
222    173
233    164
433    156
322    126
dtype: int64

### Filtering on RFM segments

In [18]:
# select bottom RFM segment "111" and view top 5  rows
datamart[datamart['RFM_Segment']=='111'][:5]

,Recency,Frequency,MonetoryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,


### Summary metrics per RFM Score

In [19]:
datamart.groupby('RFM_Score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetoryValue': ['mean', 'count']
}).round(1)

Recency Frequency MonetoryValue      
             mean      mean          mean count
RFM_Score                                      
3.0         264.8       7.8         109.1   392
4.0         174.5      13.9         227.1   391
5.0         153.0      21.2         346.8   517
6.0          94.3      28.5         491.8   468
7.0          78.8      39.7         724.2   447
8.0          62.7      57.0         974.7   467
9.0          44.2      79.0        1369.6   411
10.0         31.3     115.3        1894.0   440
11.0         20.5     193.9        3845.7   368
12.0          6.7     371.8        8850.7   471

## Grouping into named segments
Use RFM score to group custoomer into GOLD, SILVER, and Bronze segments

In [20]:
def segment_me(df):
    if df['RFM_Score'] >=9:
        return 'Gold'
    elif (df['RFM_Score'] >= 5) and (df['RFM_Score'] < 9):
        return 'Silver'
    else:
        return 'Bronze'

In [21]:
# apply segment function to datamart
datamart['General_Segment'] = datamart.apply(segment_me, axis=1)

In [22]:
datamart.groupby('General_Segment').agg({
    'Recency':'mean',
    'Frequency':'mean',
    'MonetoryValue':['mean','count']
}).round(1)

Recency Frequency MonetoryValue      
                   mean      mean          mean count
General_Segment                                      
Bronze            219.7      10.9         168.0   783
Gold               25.2     195.1        4130.3  1690
Silver             98.9      36.1         625.8  1899

In [23]:
datamart.describe()

,Recency,Frequency,MonetoryValue,RFM_Score
count,4372.000000,4372.000000,4372.000000,4372.000000
mean,92.047118,93.053294,1898.459701,7.501601
std,100.765435,232.471608,8219.345141,2.828144
min,1.000000,1.000000,-4287.630000,3.000000
25%,17.000000,17.000000,293.362500,5.000000
50%,50.000000,42.000000,648.075000,7.000000
75%,143.000000,102.000000,1611.725000,10.000000
max,374.000000,7983.000000,279489.020000,12.000000


In [24]:
# exporting RFM analysis updated file to for further clusture analysis
datamart.to_pickle(os.path.join("..", "datamart_rfm.pickle"))